In [3]:
import tensorflow as tf
import kagglehub
import numpy as np
import librosa
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
# ========== STEP 1: Load Models ==========
print("📦 Downloading and loading YAMNet model...")
yamnet_path = kagglehub.model_download("google/yamnet/tensorFlow2/yamnet")
yamnet_model = tf.saved_model.load(yamnet_path)
yamnet_infer = yamnet_model.signatures['serving_default']
print("✅ YAMNet model loaded from:", yamnet_path)



📦 Downloading and loading YAMNet model...
✅ YAMNet model loaded from: C:\Users\S\.cache\kagglehub\models\google\yamnet\tensorFlow2\yamnet\1


In [10]:

# Loading snoring classifier model
print("📦 Loading snoring classifier...")

try:
    classifier = tf.keras.models.load_model("snoring_classifier.h5", safe_mode=False)
    print("✅ Snoring classifier model loaded.")
except TypeError as e:
    print(f"❌ Error loading model: {e}")
    print("Attempting to fix InputLayer config...")

    from tensorflow.keras.models import load_model

    # Modify the config manually if needed
    classifier = load_model("snoring_classifier.h5", custom_objects={
        "InputLayer": lambda **kwargs: tf.keras.layers.InputLayer(**{k: v for k, v in kwargs.items() if k != "batch_shape"})
    })

    print("✅ Model loaded successfully after config fix.")


📦 Loading snoring classifier...
❌ Error loading model: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 1024], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']
Attempting to fix InputLayer config...


TypeError: Error when deserializing class 'Dense' using config={'name': 'dense', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'units': 256, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: Unknown dtype policy: 'DTypePolicy'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [5]:
# ========== STEP 2: Load Audio ==========
audio_path = 'm0.wav'  # Replace with your audio
waveform, sr = librosa.load(audio_path, sr=16000)
total_seconds = int(librosa.get_duration(y=waveform, sr=sr))
print(f"🎧 Audio loaded: {total_seconds} seconds")

🎧 Audio loaded: 649 seconds


In [6]:
# ========== STEP 3: Snoring Detection ==========
chunk_samples = sr  # 1 second = 16000 samples
timeline = []
snoring_seconds = 0

print("🔍 Running snoring detection...")
for sec in range(total_seconds):
    start = sec * chunk_samples
    end = start + chunk_samples
    chunk = waveform[start:end]

    if len(chunk) < chunk_samples:
        break

    chunk_tensor = tf.convert_to_tensor(chunk, dtype=tf.float32)
    results = yamnet_infer(waveform=chunk_tensor)
    embeddings = results['output_1']  # 1024-dim embeddings
    embedding_mean = tf.reduce_mean(embeddings, axis=0).numpy()

    prediction = classifier.predict(np.expand_dims(embedding_mean, axis=0), verbose=0)
    label = 1 if prediction[0][0] > 0.5 else 0
    timeline.append(label)
    if label == 1:
        snoring_seconds += 1

print(f"✅ Total snoring detected: {snoring_seconds} seconds")

🔍 Running snoring detection...


NameError: name 'classifier' is not defined

In [ ]:
# ========== STEP 4: OSA Pattern Detection ==========
osa_events = []
pause_count = 0
start_idx = None

for i, label in enumerate(timeline):
    if label == 0:  # Silence or non-snoring
        if start_idx is None:
            start_idx = i
        pause_count += 1
    else:
        if pause_count >= 10:  # Apnea-like pause ≥10s
            osa_events.append((start_idx, i))
        pause_count = 0
        start_idx = None

In [ ]:
# ========== STEP 5: OSA Summary ==========
print("\n🩺 OSA-Like Event Summary")
print(f"📌 Audio Duration: {len(timeline)} seconds")
print(f"📌 Total Snoring Seconds: {snoring_seconds}")
print(f"📌 Number of Apnea-Like Events: {len(osa_events)}")
for idx, (start, end) in enumerate(osa_events):
    print(f"  🔴 Event {idx+1}: {start}s to {end}s → {end - start}s pause")

ahi_estimate = (len(osa_events) / (len(timeline) / 60)) * 60
print(f"\n📊 Estimated AHI: {ahi_estimate:.2f} events/hour")

In [ ]:
# ========== STEP 6: Visualization ==========
sns.set_style("whitegrid")
plt.figure(figsize=(15, 4))
plt.title("Snoring & OSA Pattern Timeline")
plt.xlabel("Time (seconds)")
plt.ylabel("Prediction")

# Fuzzy smoothing
timeline_fuzzy = np.convolve(timeline, np.ones(3) / 3, mode='same')

# Color code: apnea = orange, snoring = red, non-snoring = blue
colors = []
for i in range(len(timeline)):
    in_apnea = any(start <= i < end for start, end in osa_events)
    if in_apnea:
        colors.append("orange")
    else:
        colors.append("red" if timeline[i] == 1 else "blue")

plt.scatter(range(len(timeline)), timeline_fuzzy, c=colors, s=10)
plt.yticks([0, 0.5, 1], ['Non-snoring', 'Threshold', 'Snoring'])
plt.ylim(0, 1.05)
plt.grid(True)
plt.tight_layout()
plt.show()